# 填充和步幅
:label:`sec_padding`

在前面的例子 :numref:`fig_correlation`中，输入的高度和宽度都为$3$，卷积核的高度和宽度都为$2$，生成的输出表征的维数为$2\times2$。
正如我们在 :numref:`sec_conv_layer`中所概括的那样，假设输入形状为$n_h\times n_w$，卷积核形状为$k_h\times k_w$，那么输出形状将是$(n_h-k_h+1) \times (n_w-k_w+1)$。
因此，卷积的输出形状取决于输入形状和卷积核的形状。

还有什么因素会影响输出的大小呢？本节我们将介绍*填充*（padding）和*步幅*（stride）。假设以下情景：
有时，在应用了连续的卷积之后，我们最终得到的输出远小于输入大小。这是由于卷积核的宽度和高度通常大于$1$所导致的。比如，一个$240 \times 240$像素的图像，经过$10$层$5 \times 5$的卷积后，将减少到$200 \times 200$像素。如此一来，原始图像的边界丢失了许多有用信息。而*填充*是解决此问题最有效的方法；
有时，我们可能希望大幅降低图像的宽度和高度。例如，如果我们发现原始的输入分辨率十分冗余。*步幅*则可以在这类情况下提供帮助。

## 填充

如上所述，在应用多层卷积时，我们常常丢失边缘像素。
由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素。
但随着我们应用许多连续卷积层，累积丢失的像素数就多了。
解决这个问题的简单方法即为*填充*（padding）：在输入图像的边界填充元素（通常填充元素是$0$）。
例如，在 :numref:`img_conv_pad`中，我们将$3 \times 3$输入填充到$5 \times 5$，那么它的输出就增加为$4 \times 4$。阴影部分是第一个输出元素以及用于输出计算的输入和核张量元素：
$0\times0+0\times1+0\times2+0\times3=0$。

![带填充的二维互相关。](../img/conv-pad.svg)
:label:`img_conv_pad`

通常，如果我们添加$p_h$行填充（**大约一半在顶部，一半在底部**）和$p_w$列填充（左侧大约一半，右侧一半），则输出形状将为

$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1)。$$

这意味着输出的高度和宽度将分别增加$p_h$和$p_w$。

在许多情况下，我们需要设置$p_h=k_h-1$和$p_w=k_w-1$，使输入和输出具有相同的高度和宽度。
这样可以在构建网络时更容易地预测每个图层的输出形状。
- 假设$k_h$是奇数，我们将在高度的两侧填充$p_h/2$行。
- 如果$k_h$是偶数，则一种可能性是在输入顶部填充$\lceil p_h/2\rceil$行，在底部填充$\lfloor p_h/2\rfloor$行。
同理，我们填充宽度的两侧。

卷积神经网络中*卷积核的高度和宽度通常为奇数*，例如1、3、5或7。
选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。

此外，使用奇数的核大小和填充大小也提供了书写上的便利。对于任何二维张量`X`，当满足：
1. 卷积核的大小是奇数；
2. 所有边的填充行数和列数相同；
3. 输出与输入具有相同高度和宽度
则可以得出：
>输出`Y[i, j]`是通过以输入`X[i, j]`为中心，与卷积核进行互相关计算得到的。

比如，在下面的例子中，我们创建一个高度和宽度为3的二维卷积层，并(**在所有侧边填充1个像素**)。给定高度和宽度为8的输入，则输出的高度和宽度也是8。


In [1]:
import torch
from torch import nn


# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
# 8-3+1*2+1=8
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

当卷积核的高度和宽度不同时，我们可以[**填充不同的高度和宽度**]，使输出和输入具有相同的高度和宽度。在如下示例中，我们使用高度为5，宽度为3的卷积核，高度和宽度两边的填充分别为2和1。


In [2]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## 步幅

在计算互相关时，卷积窗口从输入张量的左上角开始，向下、向右滑动。
在前面的例子中，我们默认每次滑动一个元素。
但是，有时候为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素。

我们将每次滑动元素的数量称为*步幅*（stride）。到目前为止，我们只使用过高度或宽度为$1$的步幅，那么如何使用较大的步幅呢？
 :numref:`img_conv_stride`是垂直步幅为$3$，水平步幅为$2$的二维互相关运算。
着色部分是输出元素以及用于输出计算的输入和内核张量元素：$0\times0+0\times1+1\times2+2\times3=8$、$0\times0+6\times1+0\times2+0\times3=6$。

可以看到，为了计算输出中第一列的第二个元素和第一行的第二个元素，卷积窗口分别向下滑动三行和向右滑动两列。但是，当卷积窗口继续向右滑动两列时，没有输出，因为输入元素无法填充窗口（除非我们添加另一列填充）。

![垂直步幅为 $3$，水平步幅为 $2$ 的二维互相关运算。](../img/conv-stride.svg)
:label:`img_conv_stride`

通常，当垂直步幅为$s_h$、水平步幅为$s_w$时，输出形状为

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

如果我们设置了$p_h=k_h-1$和$p_w=k_w-1$，则输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。
更进一步，如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为$(n_h/s_h) \times (n_w/s_w)$。

下面，我们[**将高度和宽度的步幅设置为2**]，从而将输入的高度和宽度减半。


In [3]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

接下来，看(**一个稍微复杂的例子**)。


In [4]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
# dim0 : (8+0*2-3+3)/3 =8/3 取下界 2
# dim1 : (8+1*2-5+4)/4 =9/4 取下界 2
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

为了简洁起见，当输入高度和宽度两侧的填充数量分别为$p_h$和$p_w$时，我们称之为填充$(p_h, p_w)$。当$p_h = p_w = p$时，填充是$p$。同理，当高度和宽度上的步幅分别为$s_h$和$s_w$时，我们称之为步幅$(s_h, s_w)$。特别地，当$s_h = s_w = s$时，我们称步幅为$s$。默认情况下，填充为0，步幅为1。在实践中，我们很少使用不一致的步幅或填充，也就是说，我们通常有$p_h = p_w$和$s_h = s_w$。

## 小结

* 填充可以增加输出的高度和宽度。这常用来使输出与输入具有相同的高和宽。
* 步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的$1/n$（$n$是一个大于$1$的整数）。
* 填充和步幅可用于有效地调整数据的维度。

## 练习

1. 对于本节中的最后一个示例，计算其输出形状，以查看它是否与实验结果一致。
1. 在本节中的实验中，试一试其他填充和步幅组合。
1. 对于音频信号，步幅$2$说明什么？
1. 步幅大于$1$的计算优势是什么？


[Discussions](https://discuss.d2l.ai/t/1851)


对于音频信号，步幅$2$说明什么？

---
在音频信号处理中，**步幅（stride）为 2** 是一个非常常见且重要的操作，尤其在卷积神经网络（CNN）或下采样（downsampling）中。下面我们从**含义、作用、影响和应用场景**四个方面详细解释。

---

1. **步幅为 2 的含义**

在对音频信号（通常是一维序列）应用卷积或池化操作时：

- **步幅（stride）= 2** 表示：**卷积核（或池化窗口）每次向右移动 2 个样本点**，而不是默认的 1。

举例：
假设音频信号长度为 8：  
`x = [x₀, x₁, x₂, x₃, x₄, x₅, x₆, x₇]`

使用 `kernel_size=3, stride=2` 的卷积：

- 第一次窗口：`[x₀, x₁, x₂]` → 输出 y₀
- 第二次窗口：`[x₂, x₃, x₄]` → 输出 y₁ （跳过 x₁）
- 第三次窗口：`[x₄, x₅, x₆]` → 输出 y₂
- （x₇ 可能被忽略，除非 padding）

→ 输出长度 ≈ 原长度 / 2

> 📌 **核心：stride=2 会让输出序列长度大约减半。**

---

 2. **主要功能与作用**

(1) **下采样（Downsampling）**
- 将高采样率信号转换为低采样率表示
- 减少时间维度上的分辨率，但保留主要特征
- 类似于图像中的“缩小”操作

(2) **降低计算量和内存占用**
- 序列长度减半 → 后续层的计算量显著减少
- 对长音频（如语音、音乐）尤其重要

(3) **扩大感受野（Receptive Field）**
- 每一层 stride=2，后续层能看到更长的原始上下文
- 例如：3 层 stride=2 的 CNN，最后一层每个输出点可覆盖原始信号的 $2^3 = 8$ 倍长度

(4) **抗混叠（Anti-aliasing）前提下的频带压缩**
- 理想下采样前应先**低通滤波**（防止高频混叠）
- 卷积核本身可起到**近似低通滤波**作用（如平滑核）
- 因此，带 stride 的卷积常隐含“滤波 + 下采样”双重功能

---

3. **对音频信号的具体影响**

| 方面 | 影响 |
|------|------|
| **时间分辨率** | 降低（细节可能丢失，如短瞬态） |
| **频率上限** | 根据奈奎斯特采样定理，新 Nyquist 频率为原采样率的 1/2→ 高频成分被抑制或混叠（若无抗混叠滤波） |
| **模型效率** | 显著提升（适合处理数秒甚至分钟级音频） |
| **语义抽象** | 促使网络在更高层次建模（如音素、词，而非原始波形） |

> 💡 在语音识别中，常用 stride=2 或更大来逐步将 **16kHz 波形 → 8kHz → 4kHz → ...**，最终得到适合建模的特征序列。

---

4. **典型应用场景**

🎙️ (1) **WaveNet / SampleRNN 等生成模型**
- 使用 stride=2 的反卷积（转置卷积）进行**上采样**
- 生成高保真音频

🗣️ (2) **语音识别（ASR）前端**
```python
# 示例：用 stride=2 卷积下采样
conv1 = nn.Conv1d(1, 64, kernel_size=10, stride=5)  # 16kHz → ~3.2kHz
conv2 = nn.Conv1d(64, 128, kernel_size=8, stride=4) # 进一步下采样
```

🎵 (3) **音乐信息检索（MIR）**
- 快速降低序列长度，提取节奏、和弦等高层特征

🔊 (4) **音频分类（如 ESC-50）**
- 类似图像 CNN，用 stride 逐步降维

⚠️ 注意事项

1. **不要盲目使用大 stride**  
   - 可能丢失重要瞬态信息（如鼓点、辅音爆破音）
2. **建议配合低通滤波**  
   - 或使用**平滑型卷积核**（如 Gaussian-like）减少混叠
3. **padding 要小心**  
   - stride=2 时，输出长度为 `⌊(L + 2p - k) / s⌋ + 1`，需合理设计

---

✅ 总结

> **在音频信号中，步幅（stride）= 2 意味着：**
> - **每两个样本取一个输出**，实现**时间维度下采样**
> - **功能包括**：降低计算量、扩大感受野、压缩频带、促进高层特征学习
> - **本质是“用时间分辨率换效率和抽象能力”**

这是现代音频深度学习模型（如 Wav2Vec, DeepSpeech, Whisper 等）中**不可或缺的基础操作**。